In [94]:

# import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile 
import warnings 
warnings.filterwarnings("ignore")
import torch
import torchaudio
from torchaudio import transforms
import os
import sys
import wave
import pandas as pd
import random

In [61]:

train_audios = os.listdir('../data/AMHARIC/data/train/wav')
test_audios = os.listdir('../data/AMHARIC/data/test/wav')



In [107]:
def visualize(aud):
    sig, sr = aud
    n_samples = aud.getnframes()
    time = np.linspace(0, n_samples/sr, num=n_samples)

    plt.plot(time,sig)
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    plt.title('Sound')
    plt.show()

In [62]:
def load(audio_file):
        sig, sr = torchaudio.load(audio_file)
        return (sig, sr)

In [63]:
def load_all_audio(audio_files):
    train_audios_loaded = []
    for train in audio_files:
        train_audios_loaded.append(load(f'../data/AMHARIC/data/train/wav/{train}'))
    return train_audios_loaded

load_all_audio(train_audios)

[(tensor([[-0.1288, -0.1282, -0.1288,  ..., -0.1245, -0.1233, -0.1251]]),
  16000),
 (tensor([[-0.1270, -0.1276, -0.1282,  ..., -0.1276, -0.1270, -0.1270]]),
  16000),
 (tensor([[-0.1233, -0.1233, -0.1233,  ..., -0.1276, -0.1276, -0.1276]]),
  16000),
 (tensor([[-0.1251, -0.1257, -0.1251,  ..., -0.1300, -0.1294, -0.1288]]),
  16000),
 (tensor([[-0.1245, -0.1239, -0.1245,  ..., -0.1257, -0.1257, -0.1257]]),
  16000),
 (tensor([[-0.1270, -0.1288, -0.1282,  ..., -0.1270, -0.1270, -0.1270]]),
  16000),
 (tensor([[-0.1196, -0.1196, -0.1196,  ..., -0.1227, -0.1215, -0.1227]]),
  16000),
 (tensor([[-0.1215, -0.1215, -0.1221,  ..., -0.1208, -0.1208, -0.1215]]),
  16000),
 (tensor([[-0.1294, -0.1306, -0.1294,  ..., -0.1312, -0.1312, -0.1318]]),
  16000),
 (tensor([[-0.1196, -0.1196, -0.1196,  ..., -0.1147, -0.1154, -0.1147]]),
  16000),
 (tensor([[-0.0215, -0.0212, -0.0215,  ..., -0.0181, -0.0178, -0.0179]]),
  16000),
 (tensor([[ 0.0000,  0.0000,  0.0000,  ..., -0.1288, -0.1276, -0.1300]]),
  

In [105]:
aud_data = load_all_audio(train_audios)

In [108]:

visualize(aud_data[0])

AttributeError: 'tuple' object has no attribute 'getnframes'

In [64]:
# def load_transcription():
    

In [65]:
def rechannel(loaded, new_channel = 2):
  sig, sr = loaded
  if (sig.shape[0] == new_channel):
      # Nothing to do
      return sig, sr

  resig = torch.cat([sig, sig])

  return ((resig, sr))
    



In [66]:
loaded_audios = load_all_audio(train_audios)

def rechannel_all_audio(all_loaded_audios):
    rechanneled_audios = []
    for audio in all_loaded_audios:
        rechanneled_audios.append(rechannel(audio))
    return rechanneled_audios
rechannel_all_audio(loaded_audios)


[(tensor([[-0.1288, -0.1282, -0.1288,  ..., -0.1245, -0.1233, -0.1251],
          [-0.1288, -0.1282, -0.1288,  ..., -0.1245, -0.1233, -0.1251]]),
  16000),
 (tensor([[-0.1270, -0.1276, -0.1282,  ..., -0.1276, -0.1270, -0.1270],
          [-0.1270, -0.1276, -0.1282,  ..., -0.1276, -0.1270, -0.1270]]),
  16000),
 (tensor([[-0.1233, -0.1233, -0.1233,  ..., -0.1276, -0.1276, -0.1276],
          [-0.1233, -0.1233, -0.1233,  ..., -0.1276, -0.1276, -0.1276]]),
  16000),
 (tensor([[-0.1251, -0.1257, -0.1251,  ..., -0.1300, -0.1294, -0.1288],
          [-0.1251, -0.1257, -0.1251,  ..., -0.1300, -0.1294, -0.1288]]),
  16000),
 (tensor([[-0.1245, -0.1239, -0.1245,  ..., -0.1257, -0.1257, -0.1257],
          [-0.1245, -0.1239, -0.1245,  ..., -0.1257, -0.1257, -0.1257]]),
  16000),
 (tensor([[-0.1270, -0.1288, -0.1282,  ..., -0.1270, -0.1270, -0.1270],
          [-0.1270, -0.1288, -0.1282,  ..., -0.1270, -0.1270, -0.1270]]),
  16000),
 (tensor([[-0.1196, -0.1196, -0.1196,  ..., -0.1227, -0.1215, -0

In [67]:
def resample(aud, newsr = 44100):
    sig, sr = aud

    if (sr == newsr):
      # Nothing to do
      return aud

    num_channels = sig.shape[0]
    # Resample first channel
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      # Resample the second channel and merge both channels
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))

In [68]:
sterio_audios = rechannel_all_audio(loaded_audios)

def resample_all_audio(sterio_audios):
    resampled_audios = []
    for aud in sterio_audios:
        resampled_audios.append(resample(aud))
    return resampled_audios

resample_all_audio(sterio_audios)

[(tensor([[-0.1281, -0.1451, -0.1387,  ..., -0.1258, -0.0848, -0.0332],
          [-0.1281, -0.1451, -0.1387,  ..., -0.1258, -0.0848, -0.0332]]),
  44100),
 (tensor([[-0.1263, -0.1434, -0.1376,  ..., -0.1277, -0.0858, -0.0335],
          [-0.1263, -0.1434, -0.1376,  ..., -0.1277, -0.0858, -0.0335]]),
  44100),
 (tensor([[-0.1227, -0.1393, -0.1334,  ..., -0.1337, -0.0962, -0.0438],
          [-0.1227, -0.1393, -0.1334,  ..., -0.1337, -0.0962, -0.0438]]),
  44100),
 (tensor([[-0.1245, -0.1415, -0.1358,  ..., -0.1350, -0.0970, -0.0441],
          [-0.1245, -0.1415, -0.1358,  ..., -0.1350, -0.0970, -0.0441]]),
  44100),
 (tensor([[-0.1239, -0.1403, -0.1341,  ..., -0.1040, -0.0535, -0.0074],
          [-0.1239, -0.1403, -0.1341,  ..., -0.1040, -0.0535, -0.0074]]),
  44100),
 (tensor([[-0.1263, -0.1441, -0.1388,  ..., -0.1277, -0.0857, -0.0335],
          [-0.1263, -0.1441, -0.1388,  ..., -0.1277, -0.0857, -0.0335]]),
  44100),
 (tensor([[-0.1190, -0.1350, -0.1293,  ..., -0.1016, -0.0524, -0

In [69]:
def resize(aud, max_ms = 4000):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms

    if (sig_len > max_len):
      # Truncate the signal to the given length
      sig = sig[:,:max_len]

    elif (sig_len < max_len):
      # Length of padding to add at the beginning and end of the signal
      pad_begin_len = random.randint(0, max_len - sig_len)
      pad_end_len = max_len - sig_len - pad_begin_len

      # Pad with 0s
      pad_begin = torch.zeros((num_rows, pad_begin_len))
      pad_end = torch.zeros((num_rows, pad_end_len))

      sig = torch.cat((pad_begin, sig, pad_end), 1)
      
    return (sig, sr)


In [74]:
resampled_audios = resample_all_audio(sterio_audios)

In [75]:


sigg, srr = resampled_audios[0]
num_rows, sig_len = sigg.shape
print(sig_len)
print(srr)

174989
44100


In [83]:
resampled_audios = resample_all_audio(sterio_audios)

def resize_all_audio(resampled_audios):
    resized_audios = []
    for aud in resampled_audios:
        resized_audios.append(resize(aud))
    return resized_audios
resize_all_audio(resampled_audios)

[(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  44100),
 (tensor([[-0.1263, -0.1434, -0.1376,  ..., -0.1332, -0.1331, -0.1336],
          [-0.1263, -0.1434, -0.1376,  ..., -0.1332, -0.1331, -0.1336]]),
  44100),
 (tensor([[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  44100),
 (tensor([[-0.1245, -0.1415, -0.1358,  ..., -0.1473, -0.1460, -0.1447],
          [-0.1245, -0.1415, -0.1358,  ..., -0.1473, -0.1460, -0.1447]]),
  44100),
 (tensor([[-0.1239, -0.1403, -0.1341,  ..., -0.1516, -0.1505, -0.1508],
          [-0.1239, -0.1403, -0.1341,  ..., -0.1516, -0.1505, -0.1508]]),
  44100),
 (tensor([[-0.1263, -0.1441, -0.1388,  ..., -0.1422, -0.1423, -0.1427],
          [-0.1263, -0.1441, -0.1388,  ..., -0.1422, -0.1423, -0.1427]]),
  44100),
 (tensor([[-0.1190, -0.1350, -0.1293,  ..., -0.1221, -0.1232, -0.1239],
          [-0.1190, -0.1350, -0.1293,  ..., -0.1221, -0.1232, -0.1239]]),
  44100),
 (tensor([[-0.1209, -0.1370,

In [85]:
resized_data = resize_all_audio(resampled_audios)

def augment_data(aud, shift_limit):
    sig,sr = aud
    _, sig_len = sig.shape
    shift_amt = int(random.random() * shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)

In [86]:
def augment_all_data(resized_data):
    augmented_data = []
    for aud in resized_data:
        augmented_data.append(augment_data(aud, 0.1))

    return augmented_data

augment_all_data(resized_data)

[(tensor([[-0.1251, -0.1252, -0.1251,  ..., -0.1252, -0.1253, -0.1251],
          [-0.1251, -0.1252, -0.1251,  ..., -0.1252, -0.1253, -0.1251]]),
  44100),
 (tensor([[-0.1283, -0.1282, -0.1283,  ..., -0.1283, -0.1282, -0.1282],
          [-0.1283, -0.1282, -0.1283,  ..., -0.1283, -0.1282, -0.1282]]),
  44100),
 (tensor([[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  44100),
 (tensor([[-0.2565, -0.2251, -0.1890,  ..., -0.3197, -0.3035, -0.2825],
          [-0.2565, -0.2251, -0.1890,  ..., -0.3197, -0.3035, -0.2825]]),
  44100),
 (tensor([[-0.1205, -0.1218, -0.1229,  ..., -0.1198, -0.1191, -0.1194],
          [-0.1205, -0.1218, -0.1229,  ..., -0.1198, -0.1191, -0.1194]]),
  44100),
 (tensor([[-0.0856, -0.0835, -0.0815,  ..., -0.0918, -0.0895, -0.0876],
          [-0.0856, -0.0835, -0.0815,  ..., -0.0918, -0.0895, -0.0876]]),
  44100),
 (tensor([[-0.1359, -0.1371, -0.1378,  ..., -0.1334, -0.1333, -0.1343],
          [-0.1359, -0.1371, -0.1378,  ..., -0.1334,

In [90]:
# mel spectrogram
def extract_feature(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80

    # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

    # Convert to decibels
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)



In [91]:
augmented_data = augment_all_data(resized_data)

def extract_feature_all(augmented_datas):
    extract_feature_data = []
    for aud in augmented_datas:
        extract_feature_data.append(extract_feature(aud))
    return extract_feature_data

extract_feature_all(augmented_data)

[tensor([[[ 29.8318,  29.3916,  29.5214,  ...,  28.9336,  29.6862,  29.3383],
          [ 20.8573,  20.4475,  20.5457,  ...,  20.0215,  20.7013,  20.4081],
          [ -9.2957,  -4.7737,  -6.4596,  ...,  -2.7406,  -9.1217,  -3.5442],
          ...,
          [-40.7620, -40.7620, -40.7620,  ..., -40.7620, -40.7620, -40.7620],
          [-40.7620, -40.7620, -40.7620,  ..., -40.7620, -40.7620, -40.7620],
          [-40.7620, -40.7620, -40.7620,  ..., -40.7620, -40.7620, -40.7620]],
 
         [[ 29.8318,  29.3916,  29.5214,  ...,  28.9336,  29.6862,  29.3383],
          [ 20.8573,  20.4475,  20.5457,  ...,  20.0215,  20.7013,  20.4081],
          [ -9.2957,  -4.7737,  -6.4596,  ...,  -2.7406,  -9.1217,  -3.5442],
          ...,
          [-40.7620, -40.7620, -40.7620,  ..., -40.7620, -40.7620, -40.7620],
          [-40.7620, -40.7620, -40.7620,  ..., -40.7620, -40.7620, -40.7620],
          [-40.7620, -40.7620, -40.7620,  ..., -40.7620, -40.7620, -40.7620]]]),
 tensor([[[ 28.2040,  29.362